#이 튜토리얼에서는 Convolutional Neural Networks for Sentence Classification 논문에서 제시된 CNN 기반 모델을 이용해서 감정 분석을 해보자. 일반적으로 CNN은 비전 관련 데이터를 처리할 때 사용되지만, 위 논문에서는 [1 x 2] 크기의 필터를 이용하여 bi-gram과 유사한 효과를 얻어내었다.


### Google Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pwd # 현재 경로 확인 

'/content/drive/My Drive/논문/sentimentP/dataset/Mecab-ko-for-Google-Colab'

In [ ]:
%cd /content/drive/MyDrive/논문/sentimentP/dataset

/content/drive/MyDrive/논문/sentimentP/dataset


In [ ]:
ls

Mecab-ko-for-Google-Colab/  ratings_train.txt  train_data.csv
ratings_test.txt            test_data.csv


### 필요한 패키지와 랜덤시드 설정

In [ ]:
# !pip install torchtext==0.10.1 
!pip install torch==1.8.0 torchtext==0.9.0
"""
아마 예전 코드를 실행하시면서 지금은 사라진 객체를 생성하시려다가 생기는 오류 같습니다.

torchtext 버전을 0.10.x 이나 그 이전으로 낮춰보시는 것은 어떠실까요?
pip install torchtext==0.10.1 과 같은 식으로 버전을 지정하셔서 설치하실 수 있습니다.

https://stackoverflow.com/questions/73055161/importerror-cannot-import-name-unicode-csv-reader-from-torchtext-utils

"""

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


'\n아마 예전 코드를 실행하시면서 지금은 사라진 객체를 생성하시려다가 생기는 오류 같습니다.\n\ntorchtext 버전을 0.10.x 이나 그 이전으로 낮춰보시는 것은 어떠실까요?\npip install torchtext==0.10.1 과 같은 식으로 버전을 지정하셔서 설치하실 수 있습니다.\n\nhttps://stackoverflow.com/questions/73055161/importerror-cannot-import-name-unicode-csv-reader-from-torchtext-utils\n\n'

# 전처리
* 이 모델은 앞서 설명한 것처럼 CNN의 필터를 이용하기 때문에 FastText처럼 bi-gram 생성 함수를 이용할 필요가 없다. 우리는 한글 데이터를 다루므로 토크나이저 또한 별도로 지정해야한다. 
* 여기서는 KoNLPy의 은전한닢 tokenizer를 이용한다.
* CNN 모델은 지난 번에 설명한 것처럼 배치 사이즈를 첫번째 차원으로 받기 때문에 batch_first = True 옵션을 주면 된다.

In [ ]:
# 코랩 내 다운로드 https://wikidocs.net/94600
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

fatal: destination path 'Mecab-ko-for-Google-Colab' already exists and is not an empty directory.
/content/drive/MyDrive/논문/sentimentP/dataset/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2023-01-29 17:22:46--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c5:2ef4, 2406:da00:ff00::22cd:e0db, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?response-content-disposition=attachment%3B%20filename%3D%22mecab-0.996-ko-0.9.2.tar.gz%22&response-content-encoding=None&AWSAcces

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

In [ ]:
import torch
import torchtext
from torchtext import data
from torchtext import datasets
from torchtext.legacy import data # 추가
import numpy as np
import random

SEED = 1027

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

### 문장의 길이가 필터 사이즈보다 작으면 에러가 나므로 다음과 같이 토크나이저를 수정

In [ ]:
FILTER_SIZES = [3,4,5]
def tokenizer(text):
    token = [t for t in mecab.morphs(text)]
    if len(token) < max(FILTER_SIZES):
        for i in range(0, max(FILTER_SIZES) - len(token)):
            token.append('<PAD>')
    return token

In [ ]:
TEXT = data.Field(tokenize = tokenizer, batch_first = True)
LABEL = data.LabelField(dtype = torch.float)

In [ ]:
fields = {'text': ('text',TEXT), 'label': ('label',LABEL)}
# dictionary 형식은 {csv컬럼명 : (데이터 컬럼명, Field이름)}

In [ ]:
import random
train_data, test_data = data.TabularDataset.splits(
                            path = '.',
                            train = 'train_data.csv',
                            test = 'test_data.csv',
                            format = 'csv',
                            fields = fields,  
)
train_data, valid_data = train_data.split(random_state=random.seed(SEED))

### 단어 벡터 처리
* 다음으로 단어 벡터는 전처리된 단어 벡터를 받자. 원 튜토리얼에선 glove.100d를 쓰지만 이건 한글을 지원하지 않으므로, 여기선 한글을 지원하는 fasttext.simple.300d 를 사용하겠다. 
* 그리고 사전훈련된 단어집에 없는 단어는 0으로 처리하는 걸 방지하기 위해 unk_init = torch.Tensor.normal_ 옵션을 준다.

In [ ]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data,
                max_size = MAX_VOCAB_SIZE,
                vectors = 'fasttext.simple.300d',
                unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

### 데이터 생성자 만들기
* 한글 데이터에선 오류가 발생해서 아래와 같이 sort_key = lambda x: len(x.text) 문장을 먼저 넣어줘야 오류없이 작동함

In [ ]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch = True,
    device = device)

# 모델 생성
* 입력 문장을 임베딩 시킨 후 2차원 CNN을 다음과 같이 적용
* 필터 사이즈는 [n x emb_dim]
* 이렇게 얻어진 벡터에 맥스 풀링(F.max_pool1d)을 적용한 후 ReLU 액티베이션을 적용한다.
* 다양한 사이즈의 필터를 적용하여 얻어진 벡터를 concatenate한 후 드랍아웃을 적용하고 마지막으로 Linear 층에 통과 시켜 output 을 산출한다.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

### 여러개의 CNN 레이어를 리스트 형태로 생성하기 위해 nn.ModuleList을 이용

### 사이즈 계산 함수 사용

In [ ]:
def print_shape(name, data):
    print(f'{name} has shape {data.shape}')

In [ ]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.convs = nn.ModuleList([nn.Conv2d(in_channels=1,
                                             out_channels=n_filters,
                                             kernel_size=(fs, embedding_dim))
                                   for fs in filter_sizes])
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        #print_shape('text', text)
        # text = [batch_size, sent_len]
        
        embedded = self.embedding(text)
        #print_shape('embedded', embedded)
        # embedded = [batch_size, sent_len, emb_dim]
        
        embedded = embedded.unsqueeze(1)
        #print_shape('embedded', embedded)
        # embedded = [batch_size, 1, sent_len, emb_dim]
        
        #print_shape('self.convs[0](embedded)', self.convs[0](embedded))
        # self.convs[0](embedded) = [batch_size, n_filters, sent_len-filter_sizes[n]+1, 1 ]
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        
        #print_shape('F.max_pool1d(conved[0], conved[0].shape[2])', F.max_pool1d(conved[0], conved[0].shape[2]))
        # F.max_pool1d(conved[0], conved[0].shape[2]) = [batch_size, n_filters, 1]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        cat = self.dropout(torch.cat(pooled, dim=1))
        #print_shape('cat', cat)
        # cat = [batch_size, n_filters * len(filter_size)]
        
        res = self.fc(cat)
        #print_shape('res', res)
        # res = [batch_size, output_dim]
        
        return self.fc(cat)

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

### 모델 벡터 사이즈 체크

In [ ]:
model = model.to(device)

### 파라미터 개수는?

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'모델의 파라미터 수는 {count_parameters(model):,} 개 입니다.')

모델의 파라미터 수는 7,861,201 개 입니다.


### 사전 훈련된 단어 벡터 불러오기

In [ ]:
pretrained_weight = TEXT.vocab.vectors
print(pretrained_weight.shape, model.embedding.weight.data.shape)

torch.Size([25002, 300]) torch.Size([25002, 300])


In [ ]:
model.embedding.weight.data.copy_(pretrained_weight)

tensor([[-1.1297e-01,  1.2156e+00,  6.8516e-01,  ..., -5.2034e-01,
         -1.6626e-01, -8.4676e-04],
        [ 9.0628e-01,  3.6853e-01, -6.4057e-01,  ...,  7.0754e-01,
          6.3230e-01,  3.6939e-01],
        [ 5.6857e-02, -5.1956e-02,  2.7326e-01,  ..., -6.9453e-02,
         -1.6064e-01, -9.8923e-02],
        ...,
        [ 1.6462e-01,  3.1504e-01, -2.1961e-01,  ...,  1.2704e+00,
          3.3490e-02,  6.4178e-01],
        [ 9.2339e-01,  6.3520e-01, -8.0703e-01,  ...,  4.7187e-02,
          9.4318e-01,  1.3437e+00],
        [-3.4884e-01,  1.5506e+00, -3.0697e-01,  ..., -9.4593e-01,
          5.8037e-01,  7.7458e-01]])

In [ ]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

# 모델 훈련

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [ ]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds==y).float()
    acc = correct.sum() / len(correct)
    return acc

### 훈련 함수 정의 
*  여기선 드랍아웃 안쓰지만 model.train() 사용

In [ ]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        predictions = model(batch.text).squeeze(1) # output_dim = 1
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

### 얼마나 훈련 걸리는지 체크하는 함수

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

### 훈련시켜보기

In [ ]:
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut4-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 5m 10s
	Train Loss: 0.449 | Train Acc: 78.49%
	 Val. Loss: 0.361 |  Val. Acc: 84.20%
Epoch: 02 | Epoch Time: 5m 20s
	Train Loss: 0.338 | Train Acc: 85.50%
	 Val. Loss: 0.343 |  Val. Acc: 85.13%
Epoch: 03 | Epoch Time: 5m 20s
	Train Loss: 0.285 | Train Acc: 88.30%
	 Val. Loss: 0.336 |  Val. Acc: 85.67%
Epoch: 04 | Epoch Time: 5m 19s
	Train Loss: 0.243 | Train Acc: 90.30%
	 Val. Loss: 0.346 |  Val. Acc: 85.93%
Epoch: 05 | Epoch Time: 5m 22s
	Train Loss: 0.205 | Train Acc: 91.90%
	 Val. Loss: 0.398 |  Val. Acc: 85.34%


### 테스트셋에 올려보기

In [ ]:
model.load_state_dict(torch.load('tut4-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.340 | Test Acc: 85.56%


더 훈련시키기

In [ ]:
for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut4-model.pt')
    
    print(f'Epoch: {epoch+6:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 06 | Epoch Time: 5m 22s
	Train Loss: 0.244 | Train Acc: 90.30%
	 Val. Loss: 0.351 |  Val. Acc: 85.80%
Epoch: 07 | Epoch Time: 5m 20s
	Train Loss: 0.207 | Train Acc: 91.97%
	 Val. Loss: 0.375 |  Val. Acc: 85.65%
Epoch: 08 | Epoch Time: 5m 21s
	Train Loss: 0.173 | Train Acc: 93.32%
	 Val. Loss: 0.418 |  Val. Acc: 85.61%
Epoch: 09 | Epoch Time: 5m 22s
	Train Loss: 0.147 | Train Acc: 94.45%
	 Val. Loss: 0.471 |  Val. Acc: 85.29%
Epoch: 10 | Epoch Time: 5m 24s
	Train Loss: 0.129 | Train Acc: 95.18%
	 Val. Loss: 0.515 |  Val. Acc: 85.08%


### 오버피팅 발생

In [ ]:
model.load_state_dict(torch.load('tut4-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.340 | Test Acc: 85.56%


### 성능은 이전 모델과 거의 비슷. 훈련시간은 대폭 감소